# Código para Geração dos Diagramas Tensão x Deformação das Amostras - TCC - Reuso do Aço

#### Gabriel Valverde - Eng. Civil - Poli

In [1]:
#Importando as bibliotecas necessárias
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

#Ignorando warnings para diminuir a poluição do terminal
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Lendo tabelas excel com os dados de tração (devem estar na mesma pasta do código)
# df = pd.read_csv("EnsaioTracao_dataset.xlsx")
dfp = pd.read_csv("EnsaioTracao_base.csv")

In [3]:
# Descobrindo a nomenclatura das colunas do DataFrame
dfp.columns

Index(['5_1_F', '5_1_D', '7_1_F', '7_1_D', '8_1_F', '8_1_D', '10_1_F',
       '10_1_D', '9_1_F', '9_1_D', '9_2_F', '9_2_D', '24_1_F', '24_1_D',
       '1_1_F', '1_1_D', '1_2_F', '1_2_D', '2_1_F', '2_1_D', '20_1_F',
       '20_1_D', '19_1_F', '19_1_D'],
      dtype='object')

In [4]:
#Definir os diâmetros de bitola de cada barra na ordem em que aparecem no data frame
bitolas=[6.71,6.61,6.47,7.96,12.33,12.33,12.48,15.93,15.91,15.99,16.01,23.05]

In [86]:
#Função para conversão de força para tensão
def to_tension(df,bt):

    df_t = df.copy()

    #Cálculo de áreas de cada barra
    areas = []
    for x in bt:
        area = (x**2)*np.pi/4
        areas.append(area)
    areas = np.array(areas)
    #print(areas)

    #Conversão dos valores de força do DataFrame para tensão 
    df_aux = df.iloc[:,::2] * 1000 / (areas)

    #Substituindo os valores de força do DataFrame para tensão
    df_t.iloc[:,::2] = df_aux.iloc[:,:]

    return df_t

In [8]:
dfp_t = to_tension(dfp, bitolas)

In [87]:

# Função de identificação do trecho linear e coeficientes da equação da reta
def linearize_diag (df, min_rsq=0.9996, int_size=300):
    s = len(df.columns)
    lista_1 = []
    lista_2 = []

    rsq_min = min_rsq   # R2 mínimo a ser buscado - quanto mais próximo de 1.0 mais linear o trecho 
                        # mas o programa pode não encontrar trecho tão linear, gerando erros. Não exagere.

    # Loop para buscar e registrar o trecho mais linear (maior R²) para cada par Tensão x Deformação
    for i in range(0,s,2):
        y = i
        x = y + 1
        t = len(df.iloc[:,x])
        R2  = False

        med = df.count()
        med = med.tolist()
        w1 = med[x]

        while w1 > 0 and not R2:
            w1 = w1 - 50                # Passo de busca ao longo do data frame
            w2 = w1 + int_size          # Tamanho do trecho a ser estudado (ALTERAR caso não obtenha o resultado desejado no plot)
            deform = df.iloc[w1:w2,x]
            tens = df.iloc[w1:w2,y]

            corr_matrix = np.corrcoef(deform, tens)
            corr = corr_matrix[0,1]
            R_sq = corr**2

            if R_sq >= rsq_min:          

                #print(df.columns[y], "R² = ", R_sq, "w1:w2 = ", w1, ":", w2)
                y0 = df.iloc[w1,y]
                x0 = df.iloc[w1,x]
                y1 = df.iloc[w2,y]
                x1 = df.iloc[w2,x]

                x_un = [x0,x1]
                y_un = [y0,y1]
                coeff = np.polyfit(x_un, y_un, 1)

                if coeff[1] < 0:        # Garantir que o trecho pré escoamento seja ajustado
                    red = -coeff[1]/coeff[0]
                    lista_2.append(red)
                    lista_1.append(w1)
                    lista_1.append(w2)
                    R2 = True


                
                
            
        #if not R2:
            #print(df.columns[y], "R² = Não Encontrado")
 
    return [lista_1,lista_2]


In [88]:
# Função para correção completa do diagrama, utiliza a função linearize e pode utilizar a função cut_end
def correct_diag(df, cut=False, min_rsq = 0.9996, int_size=300):

    lista_lin = linearize_diag(df, min_rsq, int_size)[0]
    lista_coefs = linearize_diag(df, min_rsq, int_size)[1]
    df_aux = df.copy()
    df_corr = df.copy() 

    s = len(df.columns)
    lista_corrigida = []

    # Corrige a curva, subtrai coeficientes
    df_aux = df.iloc[:,1::2] - lista_coefs

    df_corr.iloc[:,1::2] = df_aux.iloc[:,:]

    for i in range(0,s,2):
        y = i
        x = y + 1

        fim_F = df_corr.iloc[(lista_lin[i+1]-1):,y]
        fim_D = df_corr.iloc[(lista_lin[i+1]-1):,x]

        fim_F = fim_F.tolist()
        fim_D = fim_D.tolist()

        fim_D[0] = 0
        fim_F[0] = 0

        lista_corrigida.append(fim_F)
        lista_corrigida.append(fim_D) 
    
    cols = df.columns
    df_corr = pd.DataFrame(lista_corrigida).transpose()
    df_corr.columns = cols

    # Cortando o resíduo final do diagrama, necessário chamar nos argumentos
    if cut:
        df_corr = cut_end(df_corr)

    return df_corr

In [89]:
# Função para remoção do resíduo gráfico após o rompimento
def cut_end(df):
    df_cut = df.copy()
    s = len(df.columns)
    c = 0.85        # Parâmetro de corte em % de queda, alterar conforme desejado

    for i in range(0,s,2):
        y = i
        x = y + 1

        t = df_cut.iloc[:,i].count()
        r_indx = df_cut.loc[df_cut.iloc[:,i] == df_cut.iloc[:,i].max()].index[0]
        w1 = r_indx
        cut = False

        while w1 < t and not cut:
            w1 = w1 + 1
            w2 = w1 + 1

            d1 = df_cut.iloc[w1,y]
            d2 = df_cut.iloc[w2,y]
            
            if d2 < c*d1:        
                df_cut.iloc[w1:,y] = np.nan
                df_cut.iloc[w1:,x] = np.nan
                cut = True
    
    return df_cut

        

In [90]:
# Obtendo os dados corrigidos conforme a teoria
dfc_t = correct_diag(dfp_t, cut=True, min_rsq=0.9996, int_size=300)

In [97]:
# Função para plotar barras individualmente recebe (dataframe, 
# índice da barra desejada (0 a 11), observação a ser adicionada ao título)

def plot_bar(df, numbar=0, obs="", save=False, folder=""):
    plt.style.use('_mpl-gallery')
    barras = list(df.columns)
    ny = numbar*2
    nx = ny + 1

    plot_title = "Diagrama Tensão x Deformação - Amostra " + barras[ny][0:-2]
    fig_title = str(folder) + barras[ny][0:-2]
    if obs != "":
        plot_title = plot_title + " " + obs
        fig_title = fig_title + obs[1]
    
    #Plotando
    g = df.plot(x=barras[nx], y=barras[ny], xlabel="Deformação (mm)", \
                ylabel="Tensão (MPa)", title=plot_title, figsize=(6,3), legend=None).get_figure()
    
    if save:
        #Salvando gráfico já nomeado
        g.savefig(fig_title, bbox_inches="tight")


In [ ]:
# Loop para chamar a função acima para plotar cada uma das 12 amostras corrigidas
for x in range(12):
    plot_bar(dfc_t,x, save=True, folder="plots/")

In [ ]:
# Loop para chamar a função de plot para cada uma das 12 amostras não corrigidas
for x in range(12):
    plot_bar(dfp_t,x,"(Não Corrigido)", save=True, folder="plots/")

In [94]:
#Função para mapeamento do espectro RGB
def get_cmap(n, name='hsv'):
    return plt.cm.get_cmap(name, n)

In [100]:
# Função para plotar grupos de amostras na mesma figura, recebe(dataframe, lista com os índices das amostras a serem plotadas,
# observação com o nome do grupo de amostras, corr True para corrigidas e False para não corrigidas )

def plot_grupo (df, lista, obs="", group_name="", save=False, folder=""):

    plt.style.use('_mpl-gallery')
    fig = plt.figure(figsize=(7,5))
    ax = fig.add_subplot(111)
    barras = list(df.columns)  

    
    plot_title = "Diagrama Tensão x Deformação - " + group_name
    fig_title = str(folder) + str(group_name)
    if obs != "":
        plot_title = plot_title + " " + obs
        fig_title = fig_title + obs[1]    
    
    plt.title(plot_title)
    plt.xlabel("Deformação (mm)")
    plt.ylabel("Tensão (MPa)")

    loop = lista
    color = get_cmap(12)
    legend = []
    cor = [0,6,1,7,2,8,3,9,4,10,5,11]
    
    for x in loop:
        ny = x*2
        nx = ny + 1
        xa = barras[nx]
        ya = barras[ny]
        leg = ya[0:-2]
        legend.append(leg)
        ax.plot(df[xa], df[ya], c=color(cor[x]))
        
    ax.legend(legend)
    
    if save:
        #Salvando gráfico já nomeado
        fig.savefig(fig_title, bbox_inches="tight")
    plt.show()

In [ ]:
# Chamando a função plot_grupo para cada um dos grupos desejados

plot_grupo(dfc_t,[0,1,2,3],group_name="Amostras de 6,3 a 8mm",save=True,folder="plots/")
plot_grupo(dfp_t,[0,1,2,3],group_name="Amostras de 6,3 a 8mm",obs="(Não Corrigido)", save=True, folder="plots/")
plot_grupo(dfc_t,[4,5,6],group_name="Amostras de 12,5mm",save=True,folder="plots/")
plot_grupo(dfp_t,[4,5,6],group_name="Amostras de 12,5mm",obs="(Não Corrigido)",save=True,folder="plots/")
plot_grupo(dfc_t,[7,8,9,10],group_name="Amostras de 16mm",save=True,folder="plots/")
plot_grupo(dfp_t,[7,8,9,10],group_name="Amostras de 16mm",obs="(Não Corrigido)",save=True,folder="plots/")
plot_grupo(dfc_t,[11],group_name="Amostra de 22mm",save=True,folder="plots/")
plot_grupo(dfp_t,[11],group_name="Amostra de 22mm",obs="(Não Corrigido)",save=True,folder="plots/")
plot_grupo(dfp_t,[0,1,2,3,4,5,6,7,8,9,10,11],group_name="Visão Geral",obs="(Não Corrigido)",save=True,folder="plots/")
plot_grupo(dfc_t,[0,1,2,3,4,5,6,7,8,9,10,11],group_name="Visão Geral",save=True,folder="plots/")

In [ ]:
# Chamando a função plot_grupo para os grupos reprovados
plot_grupo(dfc_t,[0,1,2,10,11],group_name="Amostras Reprovadas em 10 ɸ",save=True,folder="plots/")
plot_grupo(dfc_t,[3,4],group_name="Amostras Reprovadas em Alongamento na Força Máxima",save=True,folder="plots/")